In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [2]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

In [3]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cum_2018_2024.csv"
df = pd.read_csv(folder_path, index_col=0)
df = df.reset_index(drop=True)

In [4]:
df['Teams']

0        BOS
1        GSW
2        OKC
3        PHI
4        ATL
        ... 
33747    SAC
33748    SAS
33749    TOR
33750    UTA
33751    WAS
Name: Teams, Length: 33752, dtype: object

In [5]:
date_prediction = '2024-04-06'
df = df[df['Date'] == '2024-04-05']
df['date_next'] = add_col(df, 'Date')
df['date_next'].fillna(date_prediction, inplace=True)
dates = df['date_next'].unique()

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_38719/138724945.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_38719/4291372892.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or

In [6]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'Los Angeles Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [8]:
# Define time zones
dates_utc = []
est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
utc_timezone = pytz.timezone('UTC')

threshold_value = pd.to_datetime('2020-06-27T03:55:00Z')

full = []
for date in dates:
    date = date[:10]
    # Create a datetime object for the given time in EST
    est_time = datetime.strptime(f'{date} 7:00 PM', '%Y-%m-%d %I:%M %p')
    est_time = est_timezone.localize(est_time)

    # Convert EST time to UTC
    utc_time = est_time.astimezone(utc_timezone)
    if utc_time >= threshold_value:
        final_time = utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        dates_utc.append(final_time)

In [9]:
KEY = 'dc34ee156925bf51d4a4c33c87d440fc'
SPORT = 'basketball_nba'
REGION = 'us'
MARKET = 'h2h'
FORMAT = 'decimal'
full = []
# adjust number of dates 
DATES = dates_utc
for DATE in DATES:
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{SPORT}/odds',
        params={
            'api_key': KEY,
            'regions': REGION,
            'markets': MARKET,
            'oddsFormat': FORMAT,
            'date': DATE,
        }
    ).json()
    for i in range(len(response['data'])):
        # [Timestamp, good team, evil team, fanduel odds good team wins, draftkings odds, caesars odds]
        #best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
        bookmakers = response['data'][i]['bookmakers']
        for index, data in enumerate(bookmakers):
            best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
            if response['data'][i]['bookmakers'][index]['title'] in ["FanDuel", "BetMGM", "DraftKings"]:
                for outcome in range(2):
                    odds = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['price']
                    current_odds = best_odds[outcome][4]
                    best_odds[outcome][0] = response['timestamp']
                    best_odds[outcome][1] = response['data'][i]['id']
                    best_odds[outcome][2] = response['data'][i]['bookmakers'][index]['title']
                    best_odds[outcome][3] = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][4] = odds
                full.extend(best_odds)

In [10]:
timestamp = []
Id = []
sportsbook = []
team = []
odds = []
for i in range(len(full)):
    timestamp.append(full[i][0])
    Id.append(full[i][1])
    sportsbook.append(full[i][2])
    team.append(full[i][3])
    odds.append(full[i][4])
df = pd.DataFrame({"Timestamp": timestamp,"Id": Id, "Sportsbook": sportsbook, "Team": team, "Odds": odds})

In [11]:
df

,Timestamp,Id,Sportsbook,Team,Odds
0,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,DraftKings,Cleveland Cavaliers,2.40
1,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,DraftKings,Los Angeles Lakers,1.56
2,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,FanDuel,Cleveland Cavaliers,2.82
3,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,FanDuel,Los Angeles Lakers,1.42
4,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,BetMGM,Cleveland Cavaliers,2.65
5,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,BetMGM,Los Angeles Lakers,1.48
6,2024-04-06T21:00:38Z,aaf4c40667de66fe6104d44af9459327,DraftKings,Brooklyn Nets,1.21
7,2024-04-06T21:00:38Z,aaf4c40667de66fe6104d44af9459327,DraftKings,Detroit Pistons,4.60
8,2024-04-06T21:00:38Z,aaf4c40667de66fe6104d44af9459327,FanDuel,Brooklyn Nets,1.22
9,2024-04-06T21:00:38Z,aaf4c40667de66fe6104d44af9459327,FanDuel,Detroit Pistons,4.60


In [12]:
grouped = df.groupby('Id')
group_sizes = grouped.size()

# Filter groups with less than 6 rows
ids_with_less_than_six_rows = group_sizes[group_sizes < 6].index.tolist()

print("Ids with less than 6 rows:")

ids_with_less_than_six_rows

Ids with less than 6 rows:


['3098c246e193f7b9dc6468b660dd6e47', '85a837d69d683f938e2de8bf20623b2c']

In [13]:
miss = []
for Id in ids_with_less_than_six_rows:
    group = df[df['Id'] == Id]
    sportsbooks = ["FanDuel", "BetMGM", "DraftKings"]
    
    # check which sportsbook is missing
    present_sportsbooks = group['Sportsbook'].unique()
    missing_sportsbooks = [sb for sb in sportsbooks if sb not in present_sportsbooks][0]

    teams = group['Team'].unique()
    for team in teams:
        missing = [None, None, None, None, float('-inf')]
        # calculate average odds 
        missing[0] = group['Timestamp'].iloc[0]
        missing[1] = Id
        missing[2] = missing_sportsbooks
        missing[3] = team
        missing[4] = round(group[group['Team'] == team]['Odds'].mean(), 2)
        miss.append(missing)
missing_df = pd.DataFrame(miss, columns=['Timestamp', 'Id', 'Sportsbook', 'Team', 'Odds'])

In [14]:
concat = pd.concat([df, missing_df], ignore_index=True)
concat = concat.sort_values(by=['Timestamp', 'Id'])
concat = concat.reset_index(drop=True)
full = concat.merge(concat,
               left_on=['Timestamp','Id', 'Sportsbook'],
               right_on = ['Timestamp','Id', 'Sportsbook'])
full = full.drop(full[full['Team_x'] == full['Team_y']].index)
full = full.reset_index(drop=True)

In [15]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,DraftKings,Cleveland Cavaliers,2.40,Los Angeles Lakers,1.56
1,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,DraftKings,Los Angeles Lakers,1.56,Cleveland Cavaliers,2.40
2,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,FanDuel,Cleveland Cavaliers,2.82,Los Angeles Lakers,1.42
3,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,FanDuel,Los Angeles Lakers,1.42,Cleveland Cavaliers,2.82
4,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,BetMGM,Cleveland Cavaliers,2.65,Los Angeles Lakers,1.48
5,2024-04-06T21:00:38Z,06d0be459ad524173cb82a9f227323ef,BetMGM,Los Angeles Lakers,1.48,Cleveland Cavaliers,2.65
6,2024-04-06T21:00:38Z,3098c246e193f7b9dc6468b660dd6e47,DraftKings,Brooklyn Nets,3.20,Sacramento Kings,1.37
7,2024-04-06T21:00:38Z,3098c246e193f7b9dc6468b660dd6e47,DraftKings,Sacramento Kings,1.37,Brooklyn Nets,3.20
8,2024-04-06T21:00:38Z,3098c246e193f7b9dc6468b660dd6e47,FanDuel,Brooklyn Nets,3.20,Sacramento Kings,1.37
9,2024-04-06T21:00:38Z,3098c246e193f7b9dc6468b660dd6e47,FanDuel,Sacramento Kings,1.37,Brooklyn Nets,3.20


In [16]:
full['Timestamp'] = pd.to_datetime(full['Timestamp'])

utc_timezone = pytz.utc
est_timezone = pytz.timezone('US/Eastern')

full['Timestamp'] = full['Timestamp'].dt.tz_convert(est_timezone)
full['Timestamp'] = full['Timestamp'].dt.strftime('%Y-%m-%d')

In [17]:
full['Team_x'] = full['Team_x'].map(mapping)
full['Team_y'] = full['Team_y'].map(mapping)

In [18]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-06,06d0be459ad524173cb82a9f227323ef,DraftKings,CLE,2.40,LAL,1.56
1,2024-04-06,06d0be459ad524173cb82a9f227323ef,DraftKings,LAL,1.56,CLE,2.40
2,2024-04-06,06d0be459ad524173cb82a9f227323ef,FanDuel,CLE,2.82,LAL,1.42
3,2024-04-06,06d0be459ad524173cb82a9f227323ef,FanDuel,LAL,1.42,CLE,2.82
4,2024-04-06,06d0be459ad524173cb82a9f227323ef,BetMGM,CLE,2.65,LAL,1.48
5,2024-04-06,06d0be459ad524173cb82a9f227323ef,BetMGM,LAL,1.48,CLE,2.65
6,2024-04-06,3098c246e193f7b9dc6468b660dd6e47,DraftKings,BRK,3.20,SAC,1.37
7,2024-04-06,3098c246e193f7b9dc6468b660dd6e47,DraftKings,SAC,1.37,BRK,3.20
8,2024-04-06,3098c246e193f7b9dc6468b660dd6e47,FanDuel,BRK,3.20,SAC,1.37
9,2024-04-06,3098c246e193f7b9dc6468b660dd6e47,FanDuel,SAC,1.37,BRK,3.20


In [19]:
fanduel = full[full['Sportsbook'] == 'FanDuel']
columns = fanduel.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Fanduel_odds_x','Team_y', 'Fanduel_odds_y']
fanduel.columns = columns
fanduel = fanduel.drop('Sportsbook', axis=1)

draftkings = full[full['Sportsbook'] == 'DraftKings']
columns = draftkings.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Draftkings_odds_x','Team_y', 'Draftkings_odds_y']
draftkings.columns = columns
draftkings = draftkings.drop('Sportsbook', axis=1)

betMGM = full[full['Sportsbook'] == 'BetMGM']
columns = betMGM.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'BetMGM_odds_x','Team_y', 'BetMGM_odds_y']
betMGM.columns = columns
betMGM = betMGM.drop('Sportsbook', axis=1)

In [20]:
merged_fd_dk = pd.merge(fanduel, draftkings, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

# Merge the result with betMGM DataFrame
final = pd.merge(merged_fd_dk, betMGM, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

In [21]:
columns = final.columns
columns = ['Timestamp', 'Id', 'Teams_x', 'Fanduel_odds_x','Teams_y', 'Fanduel_odds_y', 'Draftkings_odds_x', 'Draftkings_odds_y', 'BetMGM_odds_x', 'BetMGM_odds_y']
final.columns = columns
final = final.drop('Id', axis=1)

order = ['Timestamp', 'Teams_x', 'Fanduel_odds_x','Draftkings_odds_x', 'BetMGM_odds_x', 'Teams_y', 'Fanduel_odds_y','Draftkings_odds_y', 'BetMGM_odds_y']
final = final[order]

In [22]:
final

,Timestamp,Teams_x,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Teams_y,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y
0,2024-04-06,CLE,2.82,2.40,2.65,LAL,1.42,1.56,1.48
1,2024-04-06,LAL,1.42,1.56,1.48,CLE,2.82,2.40,2.65
2,2024-04-06,BRK,3.20,3.20,NaN,SAC,1.37,1.37,NaN
3,2024-04-06,SAC,1.37,1.37,NaN,BRK,3.20,3.20,NaN
4,2024-04-06,ATL,5.80,5.70,5.75,DEN,1.15,1.15,1.15
5,2024-04-06,DEN,1.15,1.15,1.15,ATL,5.80,5.70,5.75
6,2024-04-06,CLE,2.64,2.45,2.54,LAC,1.51,1.57,1.54
7,2024-04-06,LAC,1.51,1.57,1.54,CLE,2.64,2.45,2.54
8,2024-04-06,BRK,1.22,1.21,1.20,DET,4.60,4.60,4.60
9,2024-04-06,DET,4.60,4.60,4.60,BRK,1.22,1.21,1.20


In [23]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
whole = pd.read_csv(file_path, index_col=0)

In [24]:
complete = pd.concat([whole, final], axis=0)

In [25]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
complete.to_csv(file_path)